In [1]:
# Start with the tiny shakespeare dataset to train on
# read it in to inspect it
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115393


In [3]:
# Let's look at the first 1000 characters.
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# Here are all the unique characters that occur in this text dataset.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = {s:i for i,s in enumerate(chars)} # string character to index
itos = {i:s for i,s in enumerate(chars)} # index to string characters
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# Let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# Split up the data into train and validation sets
n = int(0.9*len(data)) # size of train set
train_data = data[:n]
val_data = data[n:]

In [11]:
# Define the context window size
context_len = 8
train_data[:context_len+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
# an example showing the x and its corresponding y
x = train_data[:context_len]
y = train_data[1:context_len+1]
for t in range(context_len):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target: {target}.")

when input is tensor([18]), the target: 47.
when input is tensor([18, 47]), the target: 56.
when input is tensor([18, 47, 56]), the target: 57.
when input is tensor([18, 47, 56, 57]), the target: 58.
when input is tensor([18, 47, 56, 57, 58]), the target: 1.
when input is tensor([18, 47, 56, 57, 58,  1]), the target: 15.
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47.
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58.


In [19]:
# batching data
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
context_len = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - context_len, (batch_size,))
    x = torch.stack([data[i:i+context_len] for i in ix])
    y = torch.stack([data[i+1:i+context_len+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("input:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range(batch_size):
    for t in range(context_len):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context} the target: {target}")

input:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
---
When input is tensor([53]) the target: 59
When input is tensor([53, 59]) the target: 6
When input is tensor([53, 59,  6]) the target: 1
When input is tensor([53, 59,  6,  1]) the target: 58
When input is tensor([53, 59,  6,  1, 58]) the target: 56
When input is tensor([53, 59,  6,  1, 58, 56]) the target: 47
When input is tensor([53, 59,  6,  1, 58, 56, 47]) the target: 40
When input is tensor([53, 59,  6,  1, 58, 56, 47, 40]) the target: 59
When input is tensor([49]) the target: 43
When input is tensor([49, 43]) the target: 43
When input is tensor([49, 43, 43]) the target: 54
When input is tensor

In [20]:
print(xb) # our input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [29]:
# implement the simplist language model: Bigram Language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = 0
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            # Bigram Language Model only attains to the last token in the context
            logits = logits[:, -1, :]
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)
ou, tribOL?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [35]:
# Train the Bigram Language Model
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
for steps in range(10000):
    xb, yb = get_batch("train")
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())

2.4477767944335938
2.414846897125244
2.534339427947998
2.441957712173462
2.4927189350128174
2.538072347640991
2.469179153442383
2.3589625358581543
2.550405979156494
2.525794506072998
2.487231969833374
2.342618942260742
2.526686668395996
2.4383652210235596
2.5807011127471924
2.6034011840820312
2.4883041381835938
2.612470865249634
2.4551663398742676
2.4276721477508545
2.5239176750183105
2.47395658493042
2.4398539066314697
2.407153844833374
2.3732731342315674
2.465796947479248
2.470717668533325
2.359301805496216
2.3485140800476074
2.468308925628662
2.4274988174438477
2.5160329341888428
2.388836622238159
2.570950508117676
2.360649347305298
2.4564015865325928
2.431567907333374
2.3932602405548096
2.385103940963745
2.4158248901367188
2.5392887592315674
2.4004769325256348
2.429645299911499
2.441772222518921
2.4341657161712646
2.436052083969116
2.4614694118499756
2.466155529022217
2.578524112701416
2.407141923904419
2.4196927547454834
2.487020492553711
2.540640354156494
2.5376298427581787
2.437

In [37]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


NCl
Have muens at seyomy shathe f co ssinth osingippsoouethouay, that'AUCENCEESCLT:
MOrow t.
ABES:
Whithe teatath, fat IARDorid co has:



PESTELIn anghistothoyo tishe gur wos, ha, h thy, bly'thed m inghe: ied wnong d s ccene hes nd s wsathat in t he al tur fe atheesofe histlod ag s; esersth bl VI hin ul thiloblishomeinde.
MIO thyan rco, areofo icouof k INI irt imedou t nd dore wheced bukns I' dro


### The mathematical trick in self-attention

In [39]:
# consider the following toy example

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [40]:
# we want the representation vector used to predict the next token is the average of the current vector and all vectors before it
# logits[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xbow[b,t] = torch.mean(x[b, :t+1], dim=0)

In [42]:
print(x[0])
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [43]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [57]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [59]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) -----> (B, T, T) @ (B, T, C) due to broadcasting -----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [66]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, value=float("-inf"))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [74]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single head performing self-attention
head_size = 16 # the dimention of query and key vectors
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) -------> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros(T,T) # initialize an affinity matrix with all zeros
wei = wei.masked_fill(tril == 0, value=float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other.
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [77]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)

In [78]:
k.var()

tensor(1.0449)

In [79]:
q.var()

tensor(1.0700)

In [80]:
wei.var()

tensor(17.4690)

If the scaling term 1/sqrt(head_size) is removed, the variance of attention matrix will be on the order of head_size, which in this case equals to 16. The variance of attention matrix will be preserved to 1 after adding the scaling term.

In [81]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [82]:
k.var()

tensor(0.9006)

In [83]:
q.var()

tensor(1.0037)

In [84]:
wei.var()

tensor(0.9957)

**Why is this so important?**

The attention matrix is fed into the Softmax function. So it's very important, especially at initialization, that the weights are fairly diffuse. The problem is that if the input matrix takes on very positive or negative values inside it, Softmax will actually converge towards one-hot vectors.

In [85]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [86]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])